# 14. 카메라 스티커앱을 개선하자!  
## 14-1.들어가며
우리가 이전에 만들었던 카메라 스티커앱으로 동영상을 처리하도록 개선하는 프로젝트진행.  
동영상은 단순이미지 처리하는 것 잉사응로 고려해야할 것이 많습니다.(피사체 움직임에 따른 예외상황, 처리속도, 화면떨림)  
이번시간목표는
  1. 동영상 쉽게 다룰 수 있는 방법 익히기
  2. 스티커앱 성능 분석하기 + 간단한 개선 방법
  3. 원하는 스펙을 수치로 지정하기
  4. 칼만 필터로 동영상 성능 개선하기  
  
## 준비물
프로젝트 디렉토리 생성
```
$ mkdir -p ~/aiffel/video_sticker_app/models
$ mkdir -p ~/aiffel/video_sticker_app/images
```
파일 다운로드  
```
$ cd ~/aiffel/video_sticker_app
$ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/new_video_sticker_app.zip
$ unzip new_video_sticker_app.zip
```

## 14-2. 프로젝트(1) moviepy로 비디오 처리하기  
우리가 쓸 동영상 처리 라이브러리는 파이썬 기반의 moviepy로 아래에서 moviepy가 성능면에서 적합한지 확인해보자  
**1. moviepy를 통해서 주피터노트북상에서 비디오를 읽고 쓰는 프로그램을 작성**

In [1]:
from moviepy.editor import VideoClip, VideoFileClip
from moviepy.editor import ipython_display
import cv2
import numpy as np
import os

샘플로 받은 video2.mp4를 moviepy로 읽고 width=640으로 축소해서 화면에 재생시키고, 그 영상을 mvpyresult.mp4파일로 저장.  
저장한 후에 2개의 파일을 열어 두개의 화면크기, 파일사이즈 비교

In [2]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# 쓰기
result_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
clip.write_videofile(result_video_path)

t:   0%|          | 0/404 [00:00<?, ?it/s, now=None]                

Moviepy - Building video /home/ssac19/aiffel/video_sticker_app/images/mvpyresult.mp4.
MoviePy - Writing audio in mvpyresultTEMP_MPY_wvf_snd.mp3
MoviePy - Done.
Moviepy - Writing video /home/ssac19/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/ssac19/aiffel/video_sticker_app/images/mvpyresult.mp4


**2. moviepy로 읽은 동영상을 numpy 형태로 변환하고 영상 밝기를 50% 어둡게 만든후 저장**

In [3]:
# 읽기
video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/video2.mp4'
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)
clip.ipython_display(fps=30, loop=True, autoplay=True, rd_kwargs=dict(logger=None))

# clip 에서 numpy 로 데이터 추출
vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)
for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

# 새 clip 만들기
dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

# 쓰기
result_video_path2 = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult2.mp4'
outclip.write_videofile(result_video_path2, fps=30)

t:  12%|█▏        | 48/403 [00:00<00:00, 452.98it/s, now=None]

Moviepy - Building video /home/ssac19/aiffel/video_sticker_app/images/mvpyresult2.mp4.
Moviepy - Writing video /home/ssac19/aiffel/video_sticker_app/images/mvpyresult2.mp4



Moviepy - Done !
Moviepy - video ready /home/ssac19/aiffel/video_sticker_app/images/mvpyresult2.mp4


**영상을 읽고 쓰는데 필요한 시간 측정, OpenCV와 비교**

In [4]:
# CASE 1 : moviepy 사용
start = cv2.getTickCount()
clip = VideoFileClip(video_path)
clip = clip.resize(width=640)

vlen = int(clip.duration*clip.fps)
video_container = np.zeros((vlen, clip.size[1], clip.size[0], 3), dtype=np.uint8)

for i in range(vlen):
    img = clip.get_frame(i/clip.fps)
    video_container[i] = (img * 0.5).astype(np.uint8)

dur = vlen / clip.fps
outclip = VideoClip(lambda t: video_container[int(round(t*clip.fps))], duration=dur)

mvpy_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/mvpyresult.mp4'
outclip.write_videofile(mvpy_video_path, fps=30)

time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] moviepy time : {time:.2f}ms')

t:  12%|█▏        | 48/403 [00:00<00:00, 461.34it/s, now=None]

Moviepy - Building video /home/ssac19/aiffel/video_sticker_app/images/mvpyresult.mp4.
Moviepy - Writing video /home/ssac19/aiffel/video_sticker_app/images/mvpyresult.mp4



Moviepy - Done !
Moviepy - video ready /home/ssac19/aiffel/video_sticker_app/images/mvpyresult.mp4
[INFO] moviepy time : 2.51ms


In [5]:
# CASE 2 : OpenCV 사용
start = cv2.getTickCount()
vc = cv2.VideoCapture(video_path)

cv_video_path = os.getenv('HOME')+'/aiffel/video_sticker_app/images/cvresult.mp4'
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
vw = cv2.VideoWriter(cv_video_path, fourcc, 30, (640,360))

vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))

for i in range(vlen):
    ret, img = vc.read()
    if ret == False: break
    
    img_result = cv2.resize(img, (640, 360)) * 0.5
    vw.write(img_result.astype(np.uint8))
    
time = (cv2.getTickCount() - start) / cv2.getTickFrequency()
print (f'[INFO] cv time : {time:.2f}ms')

[INFO] cv time : 1.58ms


**4. moviepy 를 이용할 때의 장단점을 분석해 봅시다. 주피터 노트북에 답변을 작성해 코드와 함께 제출해 주세요.**  
ANS) moviepy는 numpy와 호환이 되며 간단하고 직관적인 면이 장점인것 같습니다.

## 14-3. 프로젝트(2) 어디까지 만들고 싶은지 정의하기  
### 1. 실시간 카메라 스티커앱을 만들어 봅시다
이전노드에서 img2sticker_orig라는 메소드를 구현하여 addsticker.py에 저장.  
해당 메소드를 그대로 복사해서 img2sticker라는 메소드를 만들고, newaddsticker.py에 저장.  
오늘은 이 메소드를 보완, 구현하는 방향으로 진행  

**CASE 1: 웹캠입력을 사용하는 경우**  
cv2.VideoCapture(0)으로 웹캠 입력을 받아온다.
현재의 기본적인 img2sticker 를 활용하여 가장 기본적인 웹캠 실시간 스티커앱을 아래와 같이 빠르게 만들어 보겠습니다.
위에서 0은 시스템에 연결된 영상 입력장치의 인덱스로 보통 캡처할 장치가 하나(웹캠)이기에 0을 사용한다.  
```
#webcam_sticker.py
import numpy as np
import cv2
import dlib

from newaddsticker import img2sticker

detector_hog = dlib.get_frontal_face_detector()
landmark_predictor = dlib.shape_predictor('./models/shape_predictor_68_face_landmarks.dat')

def main():
    cv2.namedWindow('show', 0)
    cv2.resizeWindow('show', 640, 360)

    vc = cv2.VideoCapture(0)   # 연결된 영상 장치의 인덱스, 하나만 있는 경우 0을 사용
    img_sticker = cv2.imread('./images/king.png')

    vlen = int(vc.get(cv2.CAP_PROP_FRAME_COUNT))
    print (vlen) # 웹캠은 video length 가 0 입니다.

    # 정해진 길이가 없기 때문에 while 을 주로 사용합니다.
    # for i in range(vlen):
    while True:
        ret, img = vc.read()
        if ret == False:
            break
        start = cv2.getTickCount()
        img = cv2.flip(img, 1)  # 보통 웹캠은 좌우 반전

        # 스티커 메소드를 사용
        img_result = img2sticker(img, img_sticker.copy(), detector_hog, landmark_predictor)   

        time = (cv2.getTickCount() - start) / cv2.getTickFrequency() * 1000
        print ('[INFO] time: %.2fms'%time)

        cv2.imshow('show', img_result)
        key = cv2.waitKey(1)
        if key == 27:
            break


if __name__ == '__main__':
    main()
```
아래 코드로 실행  
```
$ cd ~/aiffel/video_sticker_app && python webcam_sticker.py
```  

**CASE 2: 스마트폰 영상의 스트리밍 입력을 사용하는 경우**  
우리는 OpenCV로 처리할 수 있는 [RTMP](https://juyoung-1008.tistory.com/30)를 활용  
스마트폰에 RTMP 어플 설치하고 송출할 url을 지정하고 송출.  
스트리밍을 할 경우는  
```
#webcam_sticker.py의 cv2.VideoCapture()을 수정    
vc = cv2.VideoCapture('rtmp://rtmp.streamaxia.com/streamaxia/XXXXXX')   # XXXXXX 부분은 본인 어플리케이션에서 확인한 코드로 대체해 주세요. 
```
다시 실행해보자  
```
$ cd ~/aiffel/video_sticker_app && python webcam_sticker.py
```  
### 2. 스티커앱을 실행하고 카메라를 고정하고 서서히 멀어져봅니다. 혹은 아주 가까이 다가가 봅니다. 얼굴을 찾지 못하는 거리를 기록해주세요.  
일반적으로 약 15cm ~ 1m 30cm 범위 사이에서 얼굴 인식이 가능하다고 합니다. 실제로 측정했을 때 어떠한지 결과를 기록해 주세요.
ANS) 15cm ~ 50cm  
### 3. 다시 자리로 돌아온 후 고개를 상하좌우로 움직여주세요. yaw, pitch, roll 각도의 개념을 직접 실험해 보고 각각 몇 도까지 정상적으로 스티커앱이 동작하는지 기록해주세요.  
(예시)
yaw : y축 기준 회전 → 높이 축(-45 ~ 45도)
picth : x축 기준 회전 → 좌우 축(-20 ~ 30도)
roll : z축 기준 회전 → 거리 축(-45 ~ 45도)  
실제 측정한 결과  
yaw = -45 ~ 45  
pitch = -30 ~ 30  
roll = -45 ~ 45  
### 4. 만들고 싶은 스티커앱의 스펙(허용 거리, 허용 인원 수, 허용 각도, 안정성)을 정해주세요.  
(예시)
거리 : 25cm ~ 1m → 너무 가까우면 스티커의 의미가 없음, 셀카봉을 들었을 때의 유효거리
인원 수 : 4명 → 4인 가족 기준
허용 각도 : pitch : -20 ~ 30도, yaw : -45 ~ 45도, roll : -45 ~ 45도 → 화면을 바라볼 수 있는 각도
안정성 : 위 조건을 만족하면서 FPPI (false positive per image) 기준 < 0.003, MR (miss rate) < 1 300장당 1번 에러 = 10초=30X10에 1번 에러  
**기준의 이유를 어떻게 정했는지가 중요합니다. → 서비스 관점, 엔지니어링 관점으로 설명하면 좋습니다.**  
ANS) 원하는 스펙.
1. 거리: 요즘 핸드폰은 광각이기에 셀카봉이 없어도 된다고 판단하여 사람이 팔 길이 허용범위인 15 ~ 50cm
2. 인원: 사람팔과 광각일경우 가로로 찍으면 4명정도가 가능한 걸로 알고 있습니다.
3. 각도: yaw: =-60(고개를 돌린 경우가 많기 때문에), pitch= -10 ~ +30(보통 사진이 잘 안나오는 아래쪽에서의 셀카는 잘찍지않는다.), roll: =-30 보통 30도 내외에서 찍는 경우가 많아보입니다.
4. 안정성: 보통 핸드폰의 60hz인데, 60번당 0.5번정도 에러나는 안정성


```
Traceback (most recent call last):
  File "webcam_sticker.py", line 43, in <module>
    main()
  File "webcam_sticker.py", line 31, in main
    img_result = img2sticker(img, img_sticker.copy(), detector_hog, landmark_predictor)   
  File "/home/ssac19/aiffel/video_sticker_app/newaddsticker.py", line 42, in img2sticker
    cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)
cv2.error: OpenCV(4.5.1) /tmp/pip-req-build-hj027r8z/opencv/modules/core/src/arithm.cpp:666: error: (-209:Sizes of input arguments do not match) The operation is neither 'array op array' (where arrays have the same size and the same number of channels), nor 'array op scalar', nor 'scalar op array' in function 'arithm_op'
```
원하는 스펙.
1. 거리: 요즘 핸드폰은 광각이기에 셀카봉이 없어도 된다고 판단하여 사람이 팔 길이 허용범위인 15 ~ 50cm
2. 인원: 사람팔과 광각일경우 가로로 찍으면 4명정도가 가능한 걸로 알고 있습니다.
3. 각도: yaw: =-60(고개를 돌린 경우가 많기 때문에), pitch= -10 ~ +30(보통 사진이 잘 안나오는 아래쪽에서의 셀카는 잘찍지않는다.), roll: =-30 보통 30도 내외에서 찍는 경우가 많아보입니다.
4. 안정성: 보통 핸드폰의 60hz인데, 60번당 0.5번정도 에러나는 안정성

웹캠과 얼굴이 너무 멀어져서 얼굴인식이 안되는 경우.
yaw를 +-30도 초과시에 에러.

## 14-4. 프로젝트 (3) 스티커 Out Bound 예외처리하기  
이전 스텝의 기본 스티커앱의 문제점을 보완해보자.

### 1. 지금까지 만든 스티커앱을 이용해서 예외 상황을 찾아주세요. 특히 서서히 영상에서 좌우 경계 밖으로 나가며 코드의 행동을 확인해 보세요.  
ANS) 웹캠과 얼굴이 너무 멀어져서 얼굴인식이 안되는 경우, yaw를 +-30도 초과시에 에러.  


### 2. 문제가 어디에서 발생하는지 코드에서 확인합니다.  
(힌트) 얼굴이 카메라 왼쪽 경계를 벗어나서 detection 되는 경우 refined_x 의 값이 음수가 됩니다.
img_bgr[..., refined_x:...] 에서 numpy array의 음수 index에 접근하게 되므로 예외가 발생합니다.
newaddsticker.py의 img2sticker 메소드에서 아래 부분을 수정해 주어야 합니다.
```
### (이전 생략) ###

# sticker
img_sticker = cv2.resize(img_sticker, (w,h), interpolation=cv2.INTER_NEAREST)

refined_x = x - w // 2
refined_y = y - h

if refined_y < 0:
    img_sticker = img_sticker[-refined_y:]
    refined_y = 0

###
# TODO : x 축 예외처리 코드 추가
###

img_bgr = img_orig.copy()
sticker_area = img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]]

img_bgr[refined_y:refined_y+img_sticker.shape[0], refined_x:refined_x+img_sticker.shape[1]] = \
    cv2.addWeighted(sticker_area, 1.0, img_sticker, 0.7, 0)

return img_bgr
```
-------------------------------------------------------
### 3. Out bound 오류(경계 밖으로 대상이 나가서 생기는 오류)를 해결해 주세요.  
위 예외처리 코드 부분에 들어가야 하는 코드는 아래와 같습니다. newaddsticker.py 파일을 수정해 주세요.  
```
    if refined_x < 0:
        img_sticker = img_sticker[:, -refined_x:]
        refined_x = 0
    elif refined_x + img_sticker.shape[1] >= img_orig.shape[1]:
        img_sticker = img_sticker[:, :-(img_sticker.shape[1]+refined_x-img_orig.shape[1])]
```

### 4. 다른 예외는 어떤 것들이 있는지 정의해 주세요. 어떤 것이 문제가 되는지 스스로 정해봅시다.  
꼼꼼이 찾아보면 위에서 수정한 것과 같은 명백한 오류 발생 케이스 이외에도 다양한 예외 상황을 찾아볼 수 있을 것입니다.

정확한 정답은 없습니다. 본인이 정의하기 나름입니다.
저는 '고개를 왼쪽아래로 향하게 할 때 스티커의 모양이 일정한 것' 을 예외 상황이라고 정의했습니다.


## 14-5. 프로젝트 (4) 스티커앱 분석 - 거리, 인원 수, 각도, 시계열 안정성  
### 1. 멀어지는 경우에 왜 스티커앱이 동작하지 않는지 분석해주세요. detection, landmark, blending 단계 중 무엇이 문제일까요?
dlib detection 이 문제입니다. 멀어지면 detector_hog 단계에서 bbox 가 출력되지 않습니다.  
```
# preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)
    # detector
    img_rgb_vga = cv2.resize(img_rgb, (640, 360))
    dlib_rects = detector_hog(img_rgb_vga, 0)
    if len(dlib_rects) < 1:
        return img_orig
```  



### 2. detector_hog의 문제를 해결하기 위해, 이미지 피라미드를 조절하여 성능을 향상시키는 간단한 방법이 있습니다. 이 방법을 활용하여 img2sticker 메소드를 간단히 고쳐 봅시다.  
```
def img2sticker(img_orig, img_sticker, detector_hog, landmark_predictor):
    # preprocess
    img_rgb = cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB)

    # detector
    img_rgb_vga = cv2.resize(img_rgb, (640, 360))
    dlib_rects = detector_hog(img_rgb_vga, 1) # <- 이미지 피라미드 수 변경
    if len(dlib_rects) < 1:
        return img_orig

    # landmark
    list_landmarks = []
    for dlib_rect in dlib_rects:
        points = landmark_predictor(img_rgb_vga, dlib_rect)
        list_points = list(map(lambda p: (p.x, p.y), points.parts()))
        list_landmarks.append(list_points)
```
수정 후에 webcam_sticker.py 를 다시한번 실행하여 스티커앱이 잘 실행되는지 확인해 봅시다.

### 3. 위에서 새롭게 시도한 방법의 문제점은 무엇인가요?
속도가 현저히 느려집니다. 기존 30ms/frame 에서 120ms/frame 으로 약 4배 느려짐 → 실시간 구동이 불가능 합니다.

### 4. 실행시간을 만족할 수 있는 방법을 찾아봅시다.
hog 디텍터 -> 딥러닝 기반 디텍터로 변경할 수 있습니다. hog 학습 단계에서 다양한 각도에 대한 hog 특징을 모두 추출해서 일반화 하기 어렵기 때문에 딥러닝 기반 검출기의 성능이 훨씬 좋습니다.

딥러닝 기반 detection 방법을 조사합니다. 아래 링크를 참고하면 도움이 될 것입니다.
[How does the OpenCV deep learning face detector work?](https://www.pyimagesearch.com/2018/02/26/face-detection-with-opencv-and-deep-learning/)

opencv 는 intel cpu 을 사용할 때 dnn 모듈이 가속화를 지원하고 있습니다. 따라서 mobilenet 과 같은 작은 backbone 모델을 사용하고 ssd 를 사용한다면 충분히 만족할 만한 시간과 성능을 얻을 수 있습니다.


### 5. 인원 수, 각도 등 각 문제에 대해서 1 - 4번을 반복해주세요. (정해진 답은 없습니다.)
자유롭게 설계해주세요. 각도 문제에 대해서는 아래 방법을 적용해볼 수 있습니다.
[Facial Landmark Detection](https://www.learnopencv.com/facial-landmark-detection/)



## 14-6. 프로젝트 (5) 칼만 필터 적용하기  
### 1.  카메라 앞에서 가만히 있을 때 스티커의 움직임을 관찰해 주세요. 어떤 문제가 발생하나요?
가만히 있어도 스티커의 크리가 일정하게 유지되지 않고, 떨리는 것처럼 보이는 현상이 발생합니다.

### 2. 이론 강의에서 배운 칼만 필터를 적용해서 스티커 움직임을 안정화시켜 주세요.
칼만 필터를 구현한 모듈인 kalman.py와 이를 이용하여 tracker를 구현한 kpkf.py를 이용하여 칼만필터를 적용한 webcam_sticker_kf.py를 함께 첨부합니다. 실행해 보면 현재는 웹캠이 아니라 샘플 동영상에 칼만필터가 적용된 것을 확인하실 수 있습니다.

동영상 입력을 웹캠으로 바꾸면 우리가 만들고 있는 웹캠에도 동일하게 적용할 수 있습니다. webcam_sticker_kf.py를 참고하여 자신만의 webcam_sticker.py를 완성해 주세요.

수고많으셨습니다. 지금까지 작성하셨던 주피터 노트북 파일과 파이썬 모듈(*.py)를 포함하여 제출해 주세요.